<a href="https://colab.research.google.com/github/MuhammadTaha01/Tweets_Sentiment_Analysis/blob/main/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
# Click on download and select kaggle cli then this key will be generated
# This will directly migrate the dataset to the google collab repo, rather than installing the whole data
!kaggle datasets download kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98% 79.0M/80.9M [00:04<00:00, 25.6MB/s]
100% 80.9M/80.9M [00:04<00:00, 18.5MB/s]


In [4]:
# Extracting the zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Dataset has been extracted')


Dataset has been extracted


In [52]:
# Importing the required dependencies

# Numerical operations and handling arrays
import numpy as np

# Data manipulation and analysis (used for handling datasets)
import pandas as pd

# Regular expressions (used for text cleaning, e.g., removing special characters)
import re

import pickle

# Natural Language Toolkit (NLTK) - used for working with human language data
from nltk.corpus import stopwords  # Importing stopwords (common words like "the", "is" that are usually removed)
from nltk.stem.porter import PorterStemmer  # Importing PorterStemmer for stemming (reducing words to their root form)

# Feature extraction (converts text data into numerical format using TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer

# Splitting dataset into training and testing sets
from sklearn.model_selection import train_test_split

# Logistic Regression model for classification tasks
from sklearn.linear_model import LogisticRegression

# Evaluating the model's performance by calculating accuracy
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print('Stopwords: ', stopwords.words('english'))

Stopwords:  ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "s

In [8]:
column_name = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_name ,encoding='ISO-8859-1')
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
twitter_data.tail()

,target,id,date,flag,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [10]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [11]:
twitter_data.shape

(1600000, 6)

In [12]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [13]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [14]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [15]:
# 0 -> For Negative Tweet
# 1 -> For Positive Tweet
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [16]:
# Stemming is the process of reducing words to their root form (or "stem") by removing suffixes.
# It helps in normalizing words so that words with similar meanings are treated as the same in Natural Language Processing (NLP).
port_stem = PorterStemmer()

In [17]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [18]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [19]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [38]:
X = twitter_data['stemmed_content'].values.astype(str)
Y = twitter_data['target'].values

In [39]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [41]:
print(X.shape, X_train.shape, X_test.shape)
print(Y.shape, y_train.shape, y_test.shape)

(1600000,) (1280000,) (320000,)
(1600000,) (1280000,) (320000,)


In [42]:
vectorizer = TfidfVectorizer()

# Fit on training text and transform training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform test data using the same vocabulary
X_test_tfidf = vectorizer.transform(X_test)

In [25]:
print(X_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [26]:
model = LogisticRegression(max_iter=1000)

In [44]:
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [45]:
X_train_prediction = model.predict(X_train_tfidf)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [46]:
print('Accuracy on the training data is: ', training_data_accuracy * 100)

Accuracy on the training data is:  79.87195312499999


In [49]:
# X_test_prediction = model.predict(X_test)
# test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print("Train features:", X_train_tfidf.shape)
print("Test features:", X_test_tfidf.shape)


Train features: (1280000, 461488)
Test features: (320000, 461488)


# New Section

In [50]:
X_test_prediction = model.predict(X_test_tfidf)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [51]:
print('Accuracy on the testing data is: ', test_data_accuracy * 100)

Accuracy on the testing data is:  77.668125


In [53]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [54]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [61]:
X_new = X_test_tfidf[900]
print(y_test[900])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet
